In [8]:
 import warnings
warnings.filterwarnings(action = 'ignore')
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import keras
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
train = pd.read_csv('/gdrive/My Drive/Titanic/train.csv')
test = pd.read_csv('/gdrive/My Drive/Titanic/test.csv')
submission = pd.read_csv('/gdrive/My Drive/Titanic/gender_submission.csv')

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


# 전처리
## Name -> 칭호만 분리


In [13]:
train.iloc[0]['Name']

'Braund, Mr. Owen Harris'

In [0]:
for ind,name in enumerate(train['Name']):
  train.loc[train.PassengerId == ind+1,'Name'] = name.split('.')[0].split(' ')[-1]

for ind,name in enumerate(test['Name']):
  test.loc[test.index == ind,'Name'] = name.split('.')[0].split(' ')[-1]


In [15]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,Mr,male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,Mrs,female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,Mr,male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,Mr,male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,Mrs,female,22.0,1,1,3101298,12.2875,NaN,S


## Cabin -> 알파벳 분리

In [0]:
for ind,cabin in enumerate(train['Cabin']):
  if type(cabin) == float:
    continue
  train.loc[train.PassengerId == ind+1,'Cabin'] = cabin[0]#알파벳만 분리

In [18]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C,C
2,3,1,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Mrs,female,35.0,1,0,113803,53.1000,C,S
4,5,0,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
for ind,cabin in enumerate(test['Cabin']):
  if type(cabin) == float:
    continue
  test.loc[test.index == ind,'Cabin'] = cabin[0]
test.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,Mr,male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,Dona,female,39.0,0,0,PC 17758,108.9000,C,C
415,1307,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,Mr,male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,Master,male,NaN,1,1,2668,22.3583,NaN,C


## object -> int

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


--> Object 타입:Name, Sex, Ticket, Cabin, Embarked

- Name 칭호별 인덱싱

In [0]:
t_set = set(train.Name)|set(test.Name)
dic_Name = {i:j for i,j in zip(t_set,[k for k in range(len(t_set))])}

In [22]:
dic_Name

{'Capt': 8,
 'Col': 17,
 'Countess': 2,
 'Don': 13,
 'Dona': 5,
 'Dr': 14,
 'Jonkheer': 10,
 'Lady': 6,
 'Major': 3,
 'Master': 1,
 'Miss': 15,
 'Mlle': 9,
 'Mme': 7,
 'Mr': 12,
 'Mrs': 4,
 'Ms': 16,
 'Rev': 11,
 'Sir': 0}

In [0]:
for k,v in dic_Name.items():
  train['Name'].replace(k,v,inplace = True)
  test['Name'].replace(k,v,inplace = True)

In [24]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,12,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,4,female,38.0,1,0,PC 17599,71.2833,C,C
2,3,1,3,15,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,4,female,35.0,1,0,113803,53.1000,C,S
4,5,0,3,12,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,11,male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,15,female,19.0,0,0,112053,30.0000,B,S
888,889,0,3,15,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,12,male,26.0,0,0,111369,30.0000,C,C


In [25]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,12,male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,4,female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,12,male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,12,male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,4,female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,12,male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,5,female,39.0,0,0,PC 17758,108.9000,C,C
415,1307,3,12,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,12,male,NaN,0,0,359309,8.0500,NaN,S


In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null int64
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB


In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null int64
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 36.0+ KB


- Sex 인덱싱


In [0]:
dic_Sex = {'female':0, 'male':1}

In [0]:
for k,v in dic_Sex.items():
  train['Sex'].replace(k,v, inplace =True)
  test['Sex'].replace(k,v,inplace=True)


In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null int64
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(7), object(3)
memory usage: 83.7+ KB


In [31]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null int64
Sex            418 non-null int64
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(6), object(3)
memory usage: 36.0+ KB


- Cabin 인덱싱

In [32]:
set_cabin = set(train.Cabin)|set(test.Cabin)
set_cabin

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', nan}

In [0]:
dic_cabin = {i:e for e,i in enumerate(set_cabin)}

In [34]:
dic_cabin

{'A': 4, 'B': 3, 'C': 2, 'D': 5, 'E': 8, 'F': 6, 'G': 1, 'T': 7, nan: 0}

In [0]:
for k,v in dic_cabin.items():
  train['Cabin'].replace(k,v,inplace = True)
  test['Cabin'].replace(k,v,inplace = True)

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null int64
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          891 non-null int64
Embarked       889 non-null object
dtypes: float64(2), int64(8), object(2)
memory usage: 83.7+ KB


In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null int64
Sex            418 non-null int64
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          418 non-null int64
Embarked       418 non-null object
dtypes: float64(2), int64(7), object(2)
memory usage: 36.0+ KB


- Embarked 인덱싱

In [38]:
set_Embarked = set(train.Embarked)|set(test.Embarked)
set_Embarked

{'C', 'Q', 'S', nan}

In [0]:
dic_Embarked = {i:e for e,i in enumerate(set_Embarked)}
for k,v in dic_Embarked.items():
  train['Embarked'].replace(k,v, inplace = True)
  test['Embarked'].replace(k,v, inplace = True)


In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null int64
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          891 non-null int64
Embarked       891 non-null int64
dtypes: float64(2), int64(9), object(1)
memory usage: 83.7+ KB


In [41]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null int64
Sex            418 non-null int64
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          418 non-null int64
Embarked       418 non-null int64
dtypes: float64(2), int64(8), object(1)
memory usage: 36.0+ KB


- Ticket 살펴보기

In [0]:
#set(test.Ticket)

In [0]:
#set(train.Ticket)

In [0]:
Ticket_cnt =pd.DataFrame([len(set(train.Ticket)), len(set(test.Ticket)), len(set(train.Ticket)|set(test.Ticket))], ['train','test','train+test'], columns = ['type_count'])

In [45]:
Ticket_cnt

,type_count
train,681
test,363
train+test,929


-------------------------------
train, test 각각의 샘플에서 Ticket번호가 겹치는 경우도 있고,
<br>
train+test에서도 같은 Ticket번호가 있다.
<br>
위에서 살펴본 바, 길이 3이 가장 짧은 길이의 Ticket 번호이다. 
<br> 
따라서 Ticket번호 길이를 모두 3으로 잘라내는 건 어떨까


---



In [0]:
for ind,ticket in enumerate(train.Ticket):
  train.loc[train.index == ind, 'Ticket'] = ticket[:3]

for ind,ticket in enumerate(test.Ticket):
  test.loc[test.index == ind, 'Ticket'] = ticket[:3]

In [47]:
Ticket_cnt['type_count(after)'] = [len(set(train.Ticket)), len(set(test.Ticket)),len( set(train.Ticket)|set(test.Ticket))]
Ticket_cnt

,type_count,type_count(after)
train,681,154
test,363,117
train+test,929,171


In [0]:
#많이 줄어들었으니, 이것도 인덱싱 해보자
set_ticket = set(train.Ticket)|set(test.Ticket)
#set_ticket

In [0]:
dic_ticket = {i:e for e,i in enumerate(set_ticket)}
#dic_ticket

In [0]:
for k,v in dic_ticket.items():
  train['Ticket'].replace(k,v, inplace = True)
  test['Ticket'].replace(k,v, inplace = True)

In [51]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null int64
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null int64
Fare           891 non-null float64
Cabin          891 non-null int64
Embarked       891 non-null int64
dtypes: float64(2), int64(10)
memory usage: 83.7 KB


In [52]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null int64
Sex            418 non-null int64
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null int64
Fare           417 non-null float64
Cabin          418 non-null int64
Embarked       418 non-null int64
dtypes: float64(2), int64(9)
memory usage: 36.0 KB


### Cabin이 B,D,E인 승객 표시 -> Cabin_s

예전 분석에 따르면,<br> Cabin이 B,D,E에 해당하는 승객의 생존율이 높았다.(대부분이 Pclass가 1이기도 했음)<br>이걸 바탕으로 Cabin_s라는 새로운 컬럼을 생성하고, 각 승객의 Cabin이 B,D,E 해당 여부(0,1)를 값으로 넣어보자.


In [53]:
#인덱스 확인
dic_cabin

{'A': 4, 'B': 3, 'C': 2, 'D': 5, 'E': 8, 'F': 6, 'G': 1, 'T': 7, nan: 0}

In [0]:
def train_cabin(x):
  if train.iloc[x].Cabin in [dic_cabin['B'],dic_cabin['D'], dic_cabin['E']]:
    return 1
  return 0

def test_cabin(x):
  if test.iloc[x].Cabin in [dic_cabin['B'],dic_cabin['D'], dic_cabin['E']]:
    return 1
  return 0
  
train['Cabin_s'] = [train_cabin(x) for x in train.index]
test['Cabin_s'] = [test_cabin(x) for x in test.index]

In [55]:
train.Cabin_s.value_counts()

0    779
1    112
Name: Cabin_s, dtype: int64

In [56]:
test.Cabin_s.value_counts()

0    378
1     40
Name: Cabin_s, dtype: int64

### Sibsp + Parch = Family

예전 분석에 따라 Sibsp과 Parch 값의 합으로 구성된 Family 컬럼을 생성해보자

In [0]:
train['Family'] = train.SibSp + train.Parch

In [58]:
train.tail(7)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_s,Family
884,885,0,3,12,1,25.0,0,0,29,7.050,0,3,0,0
885,886,0,3,4,0,39.0,0,5,1,29.125,0,1,0,5
886,887,0,2,11,1,27.0,0,0,111,13.000,0,3,0,0
887,888,1,1,15,0,19.0,0,0,2,30.000,3,3,1,0
888,889,0,3,15,0,NaN,1,2,13,23.450,0,3,0,3
889,890,1,1,12,1,26.0,0,0,107,30.000,2,2,0,0
890,891,0,3,12,1,32.0,0,0,52,7.750,0,1,0,0


In [0]:
test['Family'] = test.Parch + test.SibSp

In [60]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null int64
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null int64
Fare           891 non-null float64
Cabin          891 non-null int64
Embarked       891 non-null int64
Cabin_s        891 non-null int64
Family         891 non-null int64
dtypes: float64(2), int64(12)
memory usage: 97.6 KB


## 결측치 처리

In [61]:
#Age는 mean으로 채우기
train.Age.fillna(value = train.Age.mean(),axis = 0,inplace = True)
train.describe()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_s,Family
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,10.951740,0.647587,29.699118,0.523008,0.381594,79.438833,32.204208,0.930415,2.631874,0.125701,0.904602
std,257.353842,0.486592,0.836071,3.980984,0.477990,13.002015,1.102743,0.806057,50.355070,49.693429,1.989208,0.647601,0.331699,1.613459
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,12.000000,0.000000,22.000000,0.000000,0.000000,30.000000,7.910400,0.000000,2.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,12.000000,1.000000,29.699118,0.000000,0.000000,80.000000,14.454200,0.000000,3.000000,0.000000,0.000000
75%,668.500000,1.000000,3.000000,12.000000,1.000000,35.000000,1.000000,0.000000,132.000000,31.000000,0.000000,3.000000,0.000000,1.000000
max,891.000000,1.000000,3.000000,17.000000,1.000000,80.000000,8.000000,6.000000,170.000000,512.329200,8.000000,3.000000,1.000000,10.000000


In [62]:
test.Age.fillna(value = test.Age.mean(), axis = 0,inplace = True)
test.describe()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_s,Family
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,417.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,10.645933,0.636364,30.272590,0.447368,0.392344,76.882775,35.627188,0.808612,2.535885,0.095694,0.839713
std,120.810458,0.841838,4.156875,0.481622,12.634534,0.896760,0.981429,51.487310,55.907576,1.786365,0.685516,0.294523,1.519072
min,892.000000,1.000000,1.000000,0.000000,0.170000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,996.250000,1.000000,12.000000,0.000000,23.000000,0.000000,0.000000,24.000000,7.895800,0.000000,2.000000,0.000000,0.000000
50%,1100.500000,3.000000,12.000000,1.000000,30.272590,0.000000,0.000000,78.500000,14.454200,0.000000,3.000000,0.000000,0.000000
75%,1204.750000,3.000000,12.000000,1.000000,35.750000,1.000000,0.000000,128.000000,31.500000,0.000000,3.000000,0.000000,1.000000
max,1309.000000,3.000000,17.000000,1.000000,76.000000,8.000000,9.000000,170.000000,512.329200,8.000000,3.000000,1.000000,10.000000


In [63]:
#Fare도 mean으로 채우기
test.Fare.fillna(value = test.Fare.mean(), axis=0, inplace = True)
test.describe()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_s,Family
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,10.645933,0.636364,30.272590,0.447368,0.392344,76.882775,35.627188,0.808612,2.535885,0.095694,0.839713
std,120.810458,0.841838,4.156875,0.481622,12.634534,0.896760,0.981429,51.487310,55.840500,1.786365,0.685516,0.294523,1.519072
min,892.000000,1.000000,1.000000,0.000000,0.170000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,996.250000,1.000000,12.000000,0.000000,23.000000,0.000000,0.000000,24.000000,7.895800,0.000000,2.000000,0.000000,0.000000
50%,1100.500000,3.000000,12.000000,1.000000,30.272590,0.000000,0.000000,78.500000,14.454200,0.000000,3.000000,0.000000,0.000000
75%,1204.750000,3.000000,12.000000,1.000000,35.750000,1.000000,0.000000,128.000000,31.500000,0.000000,3.000000,0.000000,1.000000
max,1309.000000,3.000000,17.000000,1.000000,76.000000,8.000000,9.000000,170.000000,512.329200,8.000000,3.000000,1.000000,10.000000


# ML 모델 적용

In [0]:
log_clf = LogisticRegression()
nb_clf = MultinomialNB()
svm_clf = svm.SVC(probability = True)
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
gb_clf = GradientBoostingClassifier()


In [0]:
all_features =list(train.columns.drop('Survived').drop('PassengerId'))

In [79]:
all_features

['Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked',
 'Cabin_s',
 'Family']

In [0]:
from itertools import combinations
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
m_dict = {e:m for e,m in enumerate(['log_clf', 'nb_clf', 'svm_clf','dt_clf', 'rf_clf', 'gb_clf'])}#모델 이름

for i in range(1,len(all_features)):#피처 수를 점차 늘림
  for feature in list(combinations(all_features, i)):#모든 피처 조합
    feature = list(feature)
    for e,model in enumerate([log_clf, nb_clf, svm_clf,dt_clf, rf_clf, gb_clf]):
      score= cross_val_score(model, train[feature],train['Survived'], cv=10).mean()
      if score > 0.8:#cross validation  결과가 0.8 이상인 것만 저장
        x_train = train[feature]
        model.fit(x_train, train['Survived'])
        submission['Survived'] = model.predict(test[feature])
        submission.to_csv(f'/gdrive/My Drive/Titanic/results_ML/{feature}_{m_dict[e]}_{round(score,3)}.csv',index=False)
      

# 제출 결과

- Pclass Name Embarked_dt_clf_0.831.csv -> <strong>0.78947</strong> <br>
- Pclass Name Ticket Cabin_s_dt_clf_0.834.csv -> 0.78468 <br>
- Pclass Name Sex Age SibSp Ticket_gb_clf_0.843.csv -> 0.76555 <br>
- Pclass Sex Age Ticket Fare Family_rf_clf_0.836.csv -> 0.77990 <br>
- Pclass Sex Age Ticket Fare Family_gb_clf_0.842.csv -> 0.75119 <br>
- Pclass Sex Age Ticket Fare Cabin_s_gb_clf_0.826.csv -> 0.72248 <br>
- Name Ticket Fare Embarked Cabin_s Family_dt_clf_0.801.csv -> 0.74641 <br>
- Sex Age SibSp Parch Fare Cabin_gb_clf_0.826.csv -> 0.76555 <br>
- Sex Age SibSp Parch Fare Embarked_rf_clf_0.801.csv -> 0.73205 <br>
- Sex Age SibSp Parch Fare Embarked_gb_clf_0.82.csv -> 0.77990 <br>
- Sex Age SibSp Ticket Cabin_s Family_rf_clf_0.8.csv -> 0.75598 <br>
- Name Sex Age Fare Cabin Family_gb_clf_0.841.csv -> 0.77033 <br>
- Name Sex Age Fare Cabin Family_rf_clf_0.822.csv -> 0.74162 <br>
- Name Sex Age SibSp Parch Cabin_s_log_clf_0.811.csv -> 0.76076 <br>
- Name Sex Age SibSp Parch Cabin_s_rf_clf_0.809.csv -> 0.72248 <br>
- Name Sex Age SibSp Parch Cabin_s_gb_clf_0.836.csv -> 0.76555 <br>
- Name Sex Age SibSp Parch Cabin_s_dt_clf_0.808.csv -> 0.70813 <br>
- Name Sex Age Fare Cabin Family_log_clf_0.817.csv -> 0.77511 <br>
- Pclass Sex Ticket Fare Embarked Cabin_s_rf_clf_0.81.csv -> 0.76555 <br>
- Pclass Sex Ticket Fare Cabin Embarked_gb_clf_0.835.csv -> 0.77511 <br>


